# Segmenting and Clustering Neighborhoods in Toronto_Week3_2

### 1) Explore and cluster the neighborhoods in Toronto

In [63]:
#Importing required Libraries
from bs4 import BeautifulSoup
import requests
import lxml
import pandas as pd

In [64]:
wikiurl='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikihtml = requests.get(wikiurl).text
bs = BeautifulSoup(wikihtml, 'html.parser')
wikitable = bs.find('table')
wikirows = wikitable.find_all('tr')
df_details =[]
for row in wikirows:
    data = row.text.split('\n')[1:-1]
    df_details.append(data)
                    
df_main = pd.DataFrame(df_details[1:], columns=df_details[0])
df_main.shape

(288, 3)

### 2) Clean the neighborhoods in Toronto

In [65]:
#Ignore cells with 'Not assigned'.
df_main.loc[df_main['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df_main['Borough']
df_main = df_main[(df_main.Postcode != 'Not assigned') & (df_main.Borough != 'Not assigned') & (df_main.Neighbourhood != 'Not assigned')]
df_main.shape

(211, 3)

In [66]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighbourhood'].tolist()))
                    
grp = df_main.groupby(['Postcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighbourhood')

print(df2.shape)

(103, 3)


In [69]:
df2.rename(columns={'Postcode':'Postalcode'},inplace=True)
df2.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [72]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [73]:
# so we will use the csv sheet provided by Coursera as an alternative
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [87]:
#rename Column
geo_df.rename(columns={'Postcode':'Postalcode'},inplace=True)
#Merge two dataframe
geo_merged = pd.merge(geo_df, df2, on='Postalcode')
# Sequence of data
geo_data=geo_merged[['Postalcode','Borough','Neighbourhood','Latitude','Longitude']]
geo_data.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
